if (!require("BiocManager", quietly = TRUE))
    install.packages("")

In [36]:
BiocManager::install("Biostrings")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.2 (2022-10-31)

Installing package(s) 'Biostrings'

also installing the dependencies ‘bitops’, ‘zlibbioc’, ‘RCurl’, ‘GenomeInfoDbData’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘XVector’, ‘GenomeInfoDb’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [39]:
install.packages("dplyr")

also installing the dependencies ‘generics’, ‘tidyselect’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [40]:
library(stringi)
library(data.table)
library(tools)
library(data.table)
library(stringdist)
library(stringr)
library(scales)
library(ggplot2)
library(Biostrings)
library(stringr)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:ggplot2’:

    vars


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [15]:
#Set working directory 
setwd("/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio")

In [16]:
#Load data per chromosome (Anna's SNPs)
chr1 <- readRDS('data/annas_snps/snps_mx1_final.rds')
chr2 <- readRDS('data/annas_snps/snps_mx2_final.rds')
chr3 <- readRDS('data/annas_snps/snps_mx3_final.rds')
chr4 <- readRDS('data/annas_snps/snps_mx4_final.rds')
chr5 <- readRDS('data/annas_snps/snps_mx5_final.rds')
#Load indexies per chromosome (Anna's SNPs)
chr1_idx <- as.data.frame(readRDS('data/annas_snps/snps_mx1_idx.rds'))
chr2_idx <- as.data.frame(readRDS('data/annas_snps/snps_mx2_idx.rds'))
chr3_idx <- as.data.frame(readRDS('data/annas_snps/snps_mx3_idx.rds'))
chr4_idx <- as.data.frame(readRDS('data/annas_snps/snps_mx4_idx.rds'))
chr5_idx <- as.data.frame(readRDS('data/annas_snps/snps_mx5_idx.rds'))

In [42]:
#Change chromosome notation per chromosome
vcf <- as.data.frame(cbind(chr1_idx = chr1_idx$`0`, chr1)) #Combine SNPs per individual and ids for Col0
colnames(vcf)[1] <- "POS"
colnames(vcf)[2] <- "REF"
#Add ALT allele frequencies (non-reference)
uniq_values <- apply(vcf[,3:29],1,unique) #From 3(first sample) to the last ncol(vcf)
###Transform  list to the matrix to get a list of alternative alleles 
ALT <- stri_list2matrix(uniq_values, byrow=TRUE) #Chr1
#Make a dataframe of all alternative alleles
colnames(ALT) <- c("ALT1", "ALT2", "ALT3","ALT4", "ALT5")  #Number of alleles based on the number of columns
#Insert reference values to each column
for(i in 3:ncol(vcf)) {       # for-loop over columns
  vcf[ , i] <- paste(vcf$REF,vcf[ , i],sep="")}
#We have 1st allele always REF. If first allele present but alternative is lost, we will keep this site as NA
vcf <- data.frame(lapply(vcf, function(x) {
  gsub("cNA", "NA",
  gsub("aNA", "NA",
  gsub("gNA", "NA",
  gsub("tNA", "NA" , x))))
  }))
#Add alternative allele 
ALT <- as.data.frame(ALT)
vcf$ALT <- paste0(ALT$ALT1,",",ALT$ALT2,",",ALT$ALT3,",",ALT$ALT4,",", ALT$ALT5 )
##Remove NA values that represent absence of ALT allele.We are not interested in casese when reference allele is present and alt are not 
#Remove NAs
vcf$ALT <- gsub("NA,","",vcf$ALT)
vcf$ALT <- gsub(",NA,","", vcf$ALT)
vcf$ALT <- gsub(",NA","", vcf$ALT)

In [21]:
head(vcf)

,POS,REF,X22002,X9537,X9905,X10002,X6966,X10024,X6069,X6124,⋯,X9543,X10015,X1741,X8236,X9728,X9888,X9764,X22007,X6909,ALT
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,c,ct,NA,NA,cc,NA,NA,NA,NA,⋯,NA,NA,NA,cc,NA,NA,NA,NA,NA,"t,c"
2,7,a,at,NA,NA,aa,NA,NA,NA,NA,⋯,NA,NA,NA,aa,NA,NA,NA,NA,NA,"t,a"
3,9,c,ca,NA,NA,cc,cc,NA,NA,NA,⋯,NA,NA,NA,cc,NA,NA,NA,NA,NA,"a,c"
4,12,a,at,NA,NA,aa,aa,NA,NA,NA,⋯,NA,NA,NA,aa,NA,NA,NA,NA,NA,"t,a"
5,15,c,ct,NA,NA,cc,cc,NA,NA,NA,⋯,NA,NA,NA,cc,NA,NA,NA,NA,NA,"t,c"
6,19,a,at,NA,NA,aa,aa,NA,NA,NA,⋯,NA,NA,NA,aa,NA,NA,NA,NA,NA,"t,a"


In [43]:
###Make the file Hapmap like
###Add hapmap-like columns
vcf$rs<- seq(1:nrow(vcf)) #Insert numeric identifier of SNP
vcf$alleles <- paste(vcf$REF,vcf$ALT,sep=",") #Insert reference and alternative allele
vcf$alleles <- sapply(strsplit(vcf$alleles,','), function(x) toString(unique(x))) #Make alleles unique (calculate eneterez of REF+ALT unque combinations, because we are interested in variations)! Keep monomorphic sites for PI calculation
vcf$counts <- nchar(vcf$alleles)#Calculate number of characters
vcf <- vcf[vcf$counts==4,] #Filter only bi-allelic variants 4 characters: letter, del, tab, letter)
vcf$alleles <-  str_replace(vcf$alleles , ', ', '/') #Change delimeter in alleles

###Make HapMap-like file
vcf$chrom <- '1' #Change it for each chromosome 
vcf$pos <- vcf$POS
vcf$strand <- "."
vcf$center <- "."
vcf$protLSID <- "."
vcf$assayLSID <- "."
vcf$panelLSID <- "."
vcf$QCcode <- "."
vcf$assembly <- "."

###Drop unnecessary columns
vcf$CHROM <- NULL
vcf$POS <- NULL
vcf$REF <- NULL
vcf$ALT <- NULL
vcf$counts <- NULL
##Transpose columns 
vcf <- vcf %>% relocate(rs)
vcf <- vcf %>% relocate(alleles, .after =  rs)
vcf <- vcf %>% relocate(chrom, .after =  alleles)
vcf <- vcf %>% relocate(pos, .after =  chrom)
vcf <- vcf %>% relocate(strand, .after =  pos)
vcf <- vcf %>% relocate(assembly, .after =  strand)
vcf <- vcf %>% relocate(center, .after =  assembly)
vcf <- vcf %>% relocate(protLSID, .after =  center)
vcf <- vcf %>% relocate(assayLSID, .after =  protLSID)
vcf <- vcf %>% relocate(panelLSID, .after =  assayLSID)
vcf <- vcf %>% relocate(QCcode, .after =  panelLSID)


###Change small letters to big and NA to gap notation
vcf <- data.frame(lapply(vcf, function(x) {
  gsub("NA","-",
  gsub("t", "T",
  gsub("a", "A",
  gsub("c", "C",
  gsub("g", "G",x)))))
  }))
                      

#Remove unrecognized letters 
vcf_corr <- vcf[!grepl("k|m|r|w|y|n|s|b|h|v|d",vcf$alleles ),]
vcf_corr$pos <- as.integer(vcf_corr$pos) #Make position integer value 
###Write to the file
write.table(vcf_corr,"/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio/analyses/004_polymorphism_annas_snps/hmp_parsed/chr1_27acc.hmp",quote=FALSE, row.names = F, sep="\t")

